In [1]:
import numpy as np
import pandas as pd

from subprocess import check_output
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.cross_validation import  train_test_split
import time #helper libraries
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from numpy import newaxis

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from google.colab import files
files.upload()

Saving nifty.csv to nifty.csv


{'nifty.csv': b'Date/Time,Open,High,Low,Close,Volume,Open Interest\r\n2015-04-01 09:15:00,8531.1,8539,8520.1,8526.35,518425,19668625\r\n2015-04-01 09:20:00,8526.55,8526.55,8515.25,8523.05,320875,19816100\r\n2015-04-01 09:25:00,8523.7,8532.8,8519,8530.25,176950,19845125\r\n2015-04-01 09:30:00,8531,8533.95,8525.3,8526.55,110925,19889925\r\n2015-04-01 09:35:00,8526.45,8528.55,8522.7,8527,104900,19923700\r\n2015-04-01 09:40:00,8527,8527,8518.5,8519.9,109550,19942925\r\n2015-04-01 09:45:00,8518.55,8525,8512,8523.55,175775,19980175\r\n2015-04-01 09:50:00,8524.05,8529,8521.55,8528.3,165775,20038500\r\n2015-04-01 09:55:00,8527,8529,8523.8,8526,78525,20047775\r\n2015-04-01 10:00:00,8527,8530.9,8524.1,8528.8,70325,20068475\r\n2015-04-01 10:05:00,8528.8,8530.9,8524.4,8529.85,83500,20079575\r\n2015-04-01 10:10:00,8529.85,8533,8524.5,8527.55,92100,20043375\r\n2015-04-01 10:15:00,8528,8540,8526.1,8536,100925,20037525\r\n2015-04-01 10:20:00,8536.05,8541.5,8533.1,8539,129000,20033525\r\n2015-04-01 10:

In [0]:
def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

def makewindow(data, seq_len):
    dresult = []
    for index in range(len(data) - seq_len):
        dresult.append(data[index: index + seq_len])
    dresult = np.array(dresult)
    return dresult

def load_data(filename, seq_len, normalise_window):
    
    df = pd.read_csv(filename)
    df = df["Close"]
    df = df.iloc[1:]
    data = np.array(df)
    
    close_ = []    
    sequence_length = seq_len 
    dresult = makewindow(data, seq_len)
    
    if normalise_window:
        result = normalise_windows(dresult)

    result = np.array(result)
    
    row = int(round(0.9 * len(result)))
    
    x_train = result[:row, :-1]
    y_train = result[:row, -1]
    train_stock = data[:row]
    x_test = result[row:, :-1]
    y_test = result[row:, -1]
    test_stock = data[row:]
    
#     print(dresult[row:row+10,-1])
    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  
    y_train = np.reshape(y_train, (y_train.shape[0],1))
    y_test = np.reshape(y_test, (y_test.shape[0],1))
    return [x_train, y_train, train_stock, x_test, y_test, test_stock,row, data]

def predictstock(stock, y):
    predicted_list = []
    for i in range(len(y)):
        predicted_list.append((y[i] + 1) * stock[i])
    return predicted_list

In [0]:

x_train, y_train, train_stock, x_test, y_test, test_stock,row,data = load_data("nifty.csv", 10, True)

In [7]:
model = Sequential()

model.add(LSTM(
    input_dim=1,
    output_dim=50,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=1))
model.add(Activation('linear'))

model.compile(loss='mse', optimizer='rmsprop')
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 1), units=50)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, None, 50)          10400     
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 50)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 70,901
Trainable params: 70,901
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
  from ipykernel import kernelapp as app


In [8]:
model.fit(
    x_train,
    y_train,
    batch_size=128,
    nb_epoch=10,
    validation_split=0.05)

/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 47699 samples, validate on 2511 samples
Epoch 1/10
47699/47699 [==============================] - 18s 371us/step - loss: 1.7087e-05 - val_loss: 2.6333e-06
Epoch 2/10
47699/47699 [==============================] - 15s 323us/step - loss: 5.9951e-06 - val_loss: 8.0864e-07
Epoch 3/10
47699/47699 [==============================] - 15s 323us/step - loss: 3.6860e-06 - val_loss: 2.6184e-06
Epoch 4/10
47699/47699 [==============================] - 15s 325us/step - loss: 3.0824e-06 - val_loss: 6.1331e-07
Epoch 5/10
47699/47699 [==============================] - 15s 323us/step - loss: 2.7489e-06 - val_loss: 7.5288e-07
Epoch 6/10
47699/47699 [==============================] - 15s 323us/step - loss: 2.5471e-06 - val_loss: 1.4065e-06
Epoch 7/10
47699/47699 [==============================] - 15s 325us/step - loss: 2.4111e-06 - val_loss: 1.6960e-06
Epoch 8/10
47699/47699 [==============================] - 15s 322us/step - loss: 2.2950e-06 - val_loss: 3.5248e-06
Epoch 9/10
47699/47699 [=======

In [12]:
def plot_results_multiple(predicted_data, true_data,length):
    plt.plot(scaler.inverse_transform(true_data.reshape(-1, 1))[length:])
    plt.plot(scaler.inverse_transform(np.array(predicted_data).reshape(-1, 1))[length:])
    plt.show()
    
#predict lenght consecutive values from a real one
def predict_sequences_multiple(model, firstValue,length):
    prediction_seqs = []
    curr_frame = firstValue
    
    for i in range(length): 
        predicted = []        
        
        print(model.predict(curr_frame[newaxis,:,:]))
        predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
        
        curr_frame = curr_frame[0:]
        curr_frame = np.insert(curr_frame[0:], i+1, predicted[-1], axis=0)
        
        prediction_seqs.append(predicted[-1])
        
    return prediction_seqs

predict_length=5
predictions = predict_sequences_multiple(model, x_test[0], predict_length)
#print(scaler.inverse_transform(np.array(predictions).reshape(-1, 1)))
plot_results_multiple(predictions, y_test, predict_length)

[[0.00270305]]
[[0.00251641]]
[[0.00243701]]
[[0.00243376]]
[[0.0024754]]


NameError: ignored

In [14]:
predictions

[0.0027030508, 0.0025164085, 0.002437012, 0.0024337638, 0.0024753977]

In [0]:
preditions_ = model.predict(x_test)

In [18]:
preditions_.shape

(5579, 1)

In [0]:
real_stocks = predictstock(test_stock, y_test)
predicted_stocks = predictstock(test_stock,preditions_)

In [1]:
import matplotlib.pyplot as plt
plt.plot(real_stocks)
plt.plot(predicted_stocks)
plt.xlim([100,400])
plt.show()

NameError: name 'real_stocks' is not defined

In [3]:
arrar = [[12],[12],12,13]

SyntaxError: invalid syntax (<ipython-input-3-f630d55dbedc>, line 1)